## Label Frames Bounding Box

Expanded version of the label_frames tool. This allows you to specify a region of an image and save the bounding box. 

TODO:
- Add pixel_increment display
- Redo instructions display

In [1]:
import os
import cv2
import csv
import pandas as pd
import numpy as np

## Read / Write to CSV

In [2]:
def get_csv_contents(csv_path):

    csv_exists = os.path.exists(csv_path)

    if csv_exists:

        csv_contents = pd.read_csv(csv_path)

        return csv_contents.to_dict(orient='records')
    else:
        return []



LABEL_FIELDS = ['image_path', 'x', 'y', 'w', 'h']

def save_csv(filename, labels):

    with open(filename, mode='w', newline='') as csvfile:

        writer = csv.DictWriter(csvfile, fieldnames=LABEL_FIELDS)

        writer.writeheader()

        for line in labels:
            writer.writerow({LABEL_FIELDS[0]: line['image_path'], LABEL_FIELDS[1]: line['x'], LABEL_FIELDS[2]: line['y'], LABEL_FIELDS[3]: line['w'], LABEL_FIELDS[4]: line['h']})

## Make Label CSV match files in directory

This opens the labels CSV file, then adds any images that are in the image_directory but not in the label frames to the array. Labels are initialized to an empty string

TODO: Only define column names once

In [3]:
def get_labels(image_directory, label_path):

    image_files = [f for f in os.listdir(image_directory) if f.endswith('.jpg')]

    image_files.sort()

    labels = get_csv_contents(label_path)

    labelled_images = [im['image_path'] for im in labels]

    for image in image_files:
        if image not in labelled_images:
            labels.append({"image_path": image, "x": "", "y": "", "w": "", "h": ""})

    return labels

## Display Frames

This has all of the application logic. It could probably be cleaned up a bit

In [6]:
# Hacky means of making sure we don't go into infinite loop
def exit_application():
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.waitKey(1)
    cv2.waitKey(1)

LABEL_INCREMENT = 10

def display_frames(image_directory, output_file, labels):

    all_images = [im['image_path'] for im in labels]
    current_index = -1
    current_label = 0

    x = 100
    y = 100
    w = 200
    h = 200

    change_increment = 10

    while True:

        if current_index == -1:
            image_path = '../media/utility_frames/instructions.jpg'
        else:
            image_path = os.path.join(image_directory, all_images[current_index])

        if not os.path.exists(image_path):
            image_path = '../media/utility_frames/frame_not_found.jpg'

        if labels[current_index]['x'] != "" and not np.isnan(labels[current_index]['x']):
            existing_x = int(labels[current_index]['x'])
            existing_y = int(labels[current_index]['y'])
            existing_w = int(labels[current_index]['w'])
            existing_h = int(labels[current_index]['h'])
        else:
            existing_x = None
            existing_y = None
            existing_w = None
            existing_h = None


        # Read the image
        image = cv2.imread(image_path)

        if current_index >= 0:
            # Overlay text information onto the frame
            cv2.putText(image, f'F: {current_index + 1} / {len(all_images)}', (10, 50), cv2.FONT_HERSHEY_PLAIN, 1.0, (0, 255, 0), 2)
            
            if existing_x:
                cv2.addWeighted(image, 0.5, cv2.rectangle(image.copy(), (existing_x, existing_y), (existing_x + existing_w, existing_y + existing_h), (255, 0, 0), -1), 0.5, 0, image)

            cv2.addWeighted(image, 0.5, cv2.rectangle(image.copy(), (x, y), (x + w, y + h), (0, 255, 0), -1), 0.5, 0, image)
            
            
        cv2.imshow(f'Frame Labeller', image)

        # Wait for user input
        key = cv2.waitKey(0)

        # Spacebar - Save current label and move to next image
        if key == ord(' '):  

            if current_index >= 0:
                labels[current_index]['x'] = x
                labels[current_index]['y'] = y
                labels[current_index]['w'] = w
                labels[current_index]['h'] = h

                save_csv(output_file, labels)
                
            if current_index < len(all_images) - 1:
                current_index += 1
        # 'q' - Exit the application
        elif key == ord('q'):
            exit_application()
            break
        # Up Arrow - Increase the label
        elif key == 0:
            change_increment += 1
        # Down Arrow - Decrease the label
        elif key == 1:
            change_increment -= 1
        # Right Arrow - Skip to next frame without saving
        elif key == 3:
            if current_index < len(all_images) - 1:
                current_index += 1
        elif key == 2:
            if current_index > -1:
                current_index -= 1
        # I key - Move bounding box up
        elif key == 105:
            y -= change_increment
        # K key - Move bounding box down
        elif key == 107:
            y += change_increment
        # J key - Move bounding box left
        elif key == 106:
            x -= change_increment
        # L key - Move bounding box right
        elif key == 108:
            x += change_increment
        # W key - Increase bounding box height
        elif key == 119:
            h += change_increment
        # S key - Decrease bounding box height
        elif key == 115:
            h -= change_increment
        # A key - Decrease bounding box width
        elif key == 97:
            w -= change_increment
        # D key - Increase bounding box width
        elif key == 100:
            w += change_increment
        

    exit_application()

## Start Frame Labeller

In [7]:
image_directory = '../media/daytime_full_cockpit_shift_perspective/'
label_file_path = '../image_labels/bounding_box_label_test.csv'

labels = get_labels(image_directory, label_file_path)

cv2.startWindowThread()

display_frames(image_directory, label_file_path, labels)